In [1]:
import requests
import json
import pandas as pd
import re
from lxml import etree
from bs4 import BeautifulSoup
import numpy as np
import datetime
'''这个是获取HIS电子病历的爬虫，病人ID为住院号
    电子病历和另一个病案室数据库的区别是：电子病历包含所有详细信息，病案室数据库只有简表
'''

'这个是获取HIS电子病历的爬虫，病人ID为住院号\n    电子病历和另一个病案室数据库的区别是：电子病历包含所有详细信息，病案室数据库只有简表\n'

In [2]:
###输入病人ID，得到web响应的token
def get_token(patient_id):
    url = "http://172.17.0.91:8081/WebService.asmx/GetEMRBrowseToken"
    payload = "{\"strYSGH\":\"strUserCode\",\"strZYHM\":\"%s\"}" % patient_id
    #print(payload)
    headers = {
        'accept': "*/*",
        'host': "172.17.0.91:8081",
        'user-agent': "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36",
        'content-type': "application/json",
        'cache-control': "no-cache"
        }
    response = requests.request("POST", url, data=payload, headers=headers)
    token = json.loads(response.text)['d']
    return token

In [3]:
###输入patient_id,token,得到每个patient_id在医院数据存储中对应的patient_Number
def get_number(patient_id,token):
    url = "http://172.17.0.91:8081/WebService.asmx/GetZyBrxx"

    payload = "{\"strEncryptImf\":\"%s\",\"strZyhm_Input\":\"%s\",\"strCypb\":\"\"}" %(token,patient_id )
    headers = {
        'accept': "*/*",
        'host': "172.17.0.91:8081",
        'user-agent': "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36",
        'content-type': "application/json",
        'cache-control': "no-cache"
        }

    response = requests.request("POST", url, data=payload, headers=headers)
    patient_Number = json.loads(response.text)['d'].split('|')[0]
    return patient_Number

In [4]:
def get_date(patient_id, token):
    url = "http://172.17.0.91:8081/WebService.asmx/GetJZRecords_ZY"
    payload = "{\"intPatientID\":0, \"isFirstView\":true, \"strEncryptImf\":\"%s\", \"strIDCardNumber\": \"\", \"strName\": \"\", \"strSecurityNumber\": \"\", \"strTelphone\": \"\", \"strZYHM\": \"%s\"}"%(token, patient_id)
    headers = {
        'accept': "*/*",
        'host': "172.17.0.91:8081",
        'user-agent': "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36",
        'content-type': "application/json",
        'cache-control': "no-cache"
            }

    response = requests.request("POST", url, data=payload, headers=headers)
    t = json.loads(response.text)['d']
    patient_dates= []
    for i in range(len(t)):
        patient_dates.append(t[i].split('|')[0])
#     patient_date = json.loads(response.text)['d'][0].split('|')[0]
    return patient_dates

In [5]:
###获取病历文件的list及各个文件对应的number
def get_EHRlist(patient_Number):
    payload = "{\"strZyh\":\"%s\"}" % patient_Number
    url = "http://172.17.0.91:8081/WebService.asmx/GetZyBlList"
    headers = {
        'accept': "*/*",
        'host': "172.17.0.91:8081",
        'user-agent': "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36",
        'content-type': "application/json",
        'cache-control': "no-cache"
        }

    response = requests.request("POST", url, data=payload, headers=headers)
    EHR_list = json.loads(response.text)['d']
    return EHR_list

In [6]:
# 获取历史病例的list及文件地址
def getAllFileNames(date, patient_id):
    year = date.split('-')[0]
    if date.split('-')[1] in ['10','11', '12']:
        month = date.split('-')[1]
    else:
        month = date.split('-')[1][-1]
    code = patient_id + '_' + date.replace('-', '')
    url = "http://172.17.0.91:8081/WebService.asmx/GetAllFileName"
    payload = "{\"filePath\": \"X:\\\\EMRSearch\\\\Files\\\\%s年\\\\%s月\\\\%s\\\\\", \"type\": 2}"%(year, month, code)
    headers = {
            'accept': "*/*",
            'host': "172.17.0.91:8081",
            'user-agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.90 Safari/537.36",
            'content-type': "application/json",
            'cache-control': "no-cache"
                }

#response = requests.request("POST", url, data=payload.encode("utf-8"), headers=headers)
    response = requests.post(url, data= payload.encode('utf-8'), headers=headers)
    files= json.loads(response.text)['d']
    return files

In [7]:
def getAllFileNames2(date, patient_id):
    year = date.split('-')[0]
    if date.split('-')[1] in ['10','11', '12']:
        month = date.split('-')[1]
    else:
        month = date.split('-')[1][-1]
    day = date.split('-')[2]
    code = patient_id + '_' + date.replace('-', '')
    url = "http://172.17.0.91:8081/WebService.asmx/GetAllFileName"
    payload = "{\"filePath\": \"X:\\\\EMRSearch\\\\Files\\\\住院\\\\%s年\\\\%s月\\\\%s日\\\\%s\\\\\",\"type\": 2}"%(year, month, day, code)
    headers = {
            'accept': "*/*",
            'host': "172.17.0.91:8081",
            'user-agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.90 Safari/537.36",
            'content-type': "application/json",
            'cache-control': "no-cache"
                }

#response = requests.request("POST", url, data=payload.encode("utf-8"), headers=headers)
    response = requests.post(url, data= payload.encode('utf-8'), headers=headers)
    files= json.loads(response.text)['d']
    return files


In [10]:
def get_reports(number):
    url = "http://172.17.0.91:8081/WebService.asmx/GetZyBlnr"

    payload = "{\"strBlbh\":\"%s\",\"times\":1,\"charNumOfOneTime\":30000}" % number
    headers = {
        'accept': "*/*",
        'host': "172.17.0.91:8081",
        'user-agent': "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36",
        'content-type': "application/json",
        'cache-control': "no-cache",
        'postman-token': "811cc4a0-1422-6135-2fdb-d4a0420c3104"
        }
    response = requests.request("POST", url, data=payload, headers=headers)
    xmlStr = json.loads(response.text)['d']
    soup = BeautifulSoup(xmlStr,'lxml')
    TXT=soup.text
    return TXT

In [11]:
def get_historical(address):
    url = "http://172.17.0.91:8081/WebService.asmx/GetFileContent"
    payload = "{\"filePath\" : \"%s\",\"times\":1,\"charNumOfOneTime\":30000}"%address
    headers = {
            'accept': "*/*",
            'host': "172.17.0.91:8081",
            'user-agent': "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36",
            'content-type': "application/json"
#         'cache-control': "no-cache"
                }
    response = requests.request("POST", url, data=payload.encode('utf-8'), headers=headers)
    lxml = json.loads(response.text)['d']
    soup = BeautifulSoup(lxml ,'lxml')
    text = soup.text
    return text


In [37]:
import mysql.connector
import pyodbc
import sqlalchemy as sql

database_username = 'root'
database_password = 'mergeddataset'
database_ip = 'localhost'
databse_name = 'db'
#database_connection = 
engine = sql.create_engine( 'mysql+mysqlconnector://{0}:{1}@{2}/{3}'.format(database_username,database_password,database_ip,databse_name))
sqlstr=("SELECT c.姓名_Y as 姓名, 住院号_C as 住院号, 入院时间_C as 入院时间, 住院科室_C as 住院科室, c.图像数_Y as 图像数, c.年龄_Y as 年龄, c.影像结论_Y as 影响结论, 手术_C as 手术, c.检查部位_Y as 检查部位, c.检查时间_Y as 检查时间, 病名_C as 病名, 病理诊断_C as 病理诊断, c.设备_Y as 设备, 免疫组化, 病理结果, 送检科室, 部位"
" FROM image_clinic_copy c Left outer join image_path_copy p using(姓名_Y, 检查时间_Y,检查号_Y,检查部位_Y,影像结论_Y)"
"WHERE 入院时间_C between '2010-1-1' and '2010-12-31' "
"union"
" SELECT p.姓名_Y as 姓名, 住院号_C as 住院号, 入院时间_C as 入院时间, 住院科室_C as 住院科室, p.图像数_Y as 图像数, p.年龄_Y as 年龄, p.影像结论_Y as 影响结论, 手术_C as 手术, p.检查部位_Y as 检查部位, p.检查时间_Y as 检查时间, 病名_C as 病名, 病理诊断_C as 病理诊断, p.设备_Y as 设备, 免疫组化, 病理结果, 送检科室, 部位"
" FROM image_clinic_copy c right outer join image_path_copy p using(姓名_Y, 检查时间_Y,检查号_Y,检查部位_Y,影像结论_Y)"
"WHERE 入院时间_C between '2010-1-1' and '2010-12-31' ")
df_onemonth = pd.read_sql(sqlstr, engine)
array = df_onemonth['住院号'].unique()
#df = pd.read_excel('C:\\Users\\Admin\\Desktop\\onemonthtest.xml')

In [38]:
len(array)

29026

In [33]:
def function (start, stop):
    for index, patient_id in enumerate(array[start:stop]):
        lock.acquire()
        index = index + start
        lock.release()
        try:
            print(index)
            print(patient_id)
            token=get_token(patient_id)

            #当前病历
            try:
                patient_Number=get_number(patient_id,token)
            except:
                pass
            try:
                EHR_list=get_EHRlist(patient_Number)
            except:
                pass
            try:
                admission_Number = re.findall(r',.*?入院记录',EHR_list)[0].split(',')[-1].split('|')[0]
            except:
                admission_Number = ''

            try:
                discharge_Number=re.findall(r',.*?出院记录',EHR_list)[0].split(',')[-1].split('|')[0]
            except:
                discharge_Number = ''

            try:
                operation_Number=re.findall(r',.*?手术记录',EHR_list)[0].split(',')[-1].split('|')[0]
            except:
                operation_Number = ''

            #历史病历
            dates = get_date(patient_id,token)
            list_inpatient = []
            list_discharge = []
            list_operation = []
            for i in range(len(dates)):
                try:
                    files = getAllFileNames(dates[i], patient_id)
                except:
                    try:
                        files = getAllFileNames2(dates[i], patient_id)
                    except:
                        pass
                inpatient_address = ''
                discharge_address = ''
                operation_address = ''
                for i in range (len(files)):
                    if re.findall(r'.*?住院.*?病历', files[i]) != []:
                        inpatient_address = files[i].split('|')[-1].replace('\\', '\\\\')
                    if re.findall(r'.*?住院.*?病历', files[i]) == [] and re.findall(r'.*?入院记录', files[i]) != []:
                        inpatient_address = files[i].split('|')[-1].replace('\\', '\\\\')
                    if re.findall(r'.*?出院小结', files[i]) != []:
                        discharge_address = files[i].split('|')[-1].replace('\\', '\\\\')
                    if re.findall(r'.*?出院记录', files[i]) != [] and re.findall(r'.*?出院小结', files[i]) == []:
                        discharge_address = files[i].split('|')[-1].replace('\\', '\\\\')
                    if re.findall(r'.*?手术记录', files[i]) != []:
                        operation_address = files[i].split('|')[-1].replace('\\', '\\\\')

                list_inpatient.append(inpatient_address)
                list_discharge.append(discharge_address)
                list_operation.append(operation_address)

            #入院记录 为例
            if admission_Number != '':
                admission_record=get_reports(admission_Number).strip().replace(' ','')
            else:
                admission_record = ''

            if discharge_Number != '':
                discharge_record = get_reports(discharge_Number).strip().replace(' ','')
            else:
                discharge_record =  ''

            if operation_Number != '':
                operation_record=get_reports(operation_Number).strip().replace(' ','')
            else:
                operation_record =''

            if admission_record != '' and re.findall('.*姓名[:|：](.*)职业[:|：].*', admission_record) != []:
                姓名 = re.findall('.*姓名[:|：](.*)职业[:|：].*', admission_record)[0]
            else: 
                姓名 =''
            if admission_record != '' and re.findall('.*科室[:|：](.*)\n.*', admission_record) != []:
                科室 = re.findall('.*科室[:|：](.*)\n.*', admission_record)[0]
            else:
                科室 = ''
            if admission_record != '' and re.findall('住院号[:|：](.*)\n.*', admission_record)!= []:
                住院号 = re.findall('住院号[:|：](.*)\n.*', admission_record)[0]
            else:
                住院号 = ''
            if admission_record != '' and re.findall('.*职业[:|：](.*)性别[:|：].*', admission_record)!=[]:
                职业 = re.findall('.*职业[:|：](.*)性别[:|：].*', admission_record)[0]
            else:
                职业 = ''
            if admission_record != '' and re.findall('.*年龄[:|：](.*)岁住址[:|：].*', admission_record)!= []:
                年龄 = re.findall('.*年龄[:|：](.*)岁住址[:|：].*', admission_record)[0]
            else:
                年龄 = ''
            if admission_record != '':   
                try:
                    婚姻 = re.findall('.*婚姻[:|：](.*)供.*史.*者：.*', admission_record, flags = re.DOTALL)[0]
                except:
                    婚姻 = ''
            else:
                婚姻 = ''
            if admission_record != '':
                try:
                    入院日期 = re.findall('.*入院日期[:|：](.{11})', admission_record )[0]
                except:
                    入院日期 = ''
            else:
                入院日期 = ''
            if admission_record != '':
                try:
                    记录日期 = re.findall('.*记录日期[:|：](.{11})', admission_record)[0]
                except:
                    记录日期 = ''
            else:
                记录日期 = ''

            if admission_record != '':
                try:
                    主诉 = re.findall('.*主诉[:|：](.*)\n', admission_record)[0]
                except:
                    主诉 = ''
            else:
                主诉 = ''

            if admission_record != '':
                try:
                    现病史 = re.findall('.*现病史[:|：]*(.*)\n', admission_record)[0]
                except:
                     现病史 = ''
            else:
                现病史 = ''

            if admission_record != '':
                try:
                    既往史 = re.findall('.*既往史[:|：]*(.*)\n', admission_record)[0]
                except:
                    既往史 = ''
            else:
                既往史 = ''

            if admission_record != '':
                try:
                    个人史 = re.findall('.*个人史[:|：]*(.*)\n', admission_record)[0]
                except:
                    个人史 = ''
            else:
                个人史 = ''

            if re.findall('.*月经婚育史', admission_record) != []:
                try:
                    婚育史 = re.findall('.*月经婚育史[:|：]*(.*)\n', admission_record)[0]
                except:
                    try:
                        婚育史 = re.findall('.*婚育史[:|：]*(.*)\n', admission_record)[0]
                    except:
                        婚育史 = ''
            else:
                婚育史 = ''

            if admission_record != '':
                try:
                    家族史 = re.findall('.*家族史[:|：]*(.*)\n', admission_record)[0]
                except:
                    家族史 = ''
            else:
                家族史 = ''

            if admission_record != '':
                try:
                    体检 = re.findall('.*体格检查(.*?)\n\n\n', admission_record, flags = re.DOTALL)[0].replace('\n', '')
                except:
                    体检 = ''
            else:
                体检 = ''

            df_ad = pd.DataFrame([])
            dict_ad = {'姓名':姓名, '科室':科室, '住院号':住院号, '职业':职业, '年龄':年龄, '婚姻':婚姻, 
                         '入院日期':入院日期, '记录日期':记录日期, '主诉':主诉, '现病史':现病史, '既往史': 既往史, '个人史': 个人史, 
                       '家族史': 家族史, '婚育史': 婚育史, '体检':体检}
            df_ad = df_ad.append(dict_ad, ignore_index=True)


            #discharge
            if discharge_record != '':
                住院号_d = re.findall('住院号[:|：](.*)\n.*', discharge_record)[0]
            else:
                住院号_d = ''
            if discharge_record != '':
                try:
                    出院日期 = re.findall('.*出院日期[:|：]*\n{0,}(.*?)共住院.*', discharge_record)[0]
                except:
                    try:
                        出院日期 = re.findall('.*出院日期[:|：]*\n{0,}(.*?)\n.*', discharge_record)[0]
                    except:
                        出院日期 = ''
            else:
                出院日期 = ''
            if discharge_record != '':
                try:
                    初步诊断 = re.findall('.*初步诊断[:|：]*\n{0,}(.*?)\n', discharge_record)[0]
                except:
                    初步诊断 = ''
            else:
                初步诊断 = ''
            if discharge_record != '' and re.findall('.*出院情况', discharge_record) != []:
                出院情况 = re.findall('.*出院情况[:|：]*\n{0,}(.*?)\n', discharge_record)[0]
            else:
                出院情况 = ''
            if discharge_record != '':
                try:
                    出院诊断 = re.findall('.*出院诊断[:|：]*\n{0,}(.*?)\n', discharge_record)[0]
                except:
                    出院诊断 = ''
            else:
                出院诊断 = ''
            if discharge_record != '':
                出院医嘱 = re.findall('.*?出院医嘱[:|：]*\n{0,}(.*?)\n', discharge_record)[0]
            else:
                出院医嘱 = ''
            df_dis = pd.DataFrame([])
            dict_discharge = {'出院日期': 出院日期,'初步诊断': 初步诊断, 
                              '出院情况': 出院情况,'出院诊断': 出院诊断,'出院医嘱': 出院医嘱}
            df_dis = df_dis.append(dict_discharge, ignore_index=True)


            #operation 
            if operation_record == '':
                住院号_o = ''
            else:
                住院号_o = re.findall('.*住院号[:|：]*(.{8})', operation_record)[0]

            if operation_record == '' or re.findall('.*手术日期', operation_record) == []:
                手术日期 = ''
            else:
                try:
                    手术日期 = re.findall('.*手术日期[:|：]*(.{11})', operation_record)[0]
                except:
                    try: 
                        手术日期 = re.findall('.*手术日期[:|：]*(.{9})', operation_record)[0]
                    except:
                        手术日期 = ''

            if operation_record == ''or re.findall('.*术中诊断', operation_record) == [] or re.findall('.*拟施手术', operation_record) == []:
                术前诊断 = ''
            elif re.search('术中诊断',operation_record).start() < re.search('拟施手术',operation_record).start():
                try:
                    术前诊断 = re.findall('.*术前诊断[:|：]*(.*)术中诊断[:|：]*.*', operation_record)[0]
                except:
                    try:
                        术前诊断 = re.findall('.*术前诊断[:|：]*(.*)\n{0,}术中诊断[:|：]*.*', operation_record)[0]
                    except:
                        术前诊断 = ''
            else:
                try:
                    术前诊断 = re.findall('.*术前诊断[:|：]*(.*)拟施手术[:|：]*.*', operation_record)[0]
                except:
                    try:
                        术前诊断 = re.findall('.*术前诊断[:|：]*(.*)\n{0,}拟施手术[:|：]*.*', operation_record)[0]
                    except:
                        术前诊断 = ''


            if re.findall('术中诊断', operation_record) == []:
                术中诊断 = ''
            else:
                try:
                    术中诊断 = re.findall('.*术中诊断[:|：]*(.*)实施手术：.*', operation_record)[0]
                except:
                    术中诊断 = ''

            if operation_record == '' or re.findall('.*实施手术', operation_record) == []:
                实施手术 = ''    
            else:
                try:
                    实施手术 = re.findall('.*实施手术[:|：]*(.*)\n.*', operation_record)[0]
                except:
                    实施手术 = ''

            if operation_record == '':
                手术经过 = ''    
            else:
                if re.findall('失血量', operation_record) != [] and re.findall('医生签名', operation_record) != []:
                    if re.search('失血量', operation_record).start() < re.search('医生签名',operation_record).start():
                        try:
                            手术经过 = re.findall('.*手术经过(.*?)失血量.*',operation_record, flags = re.DOTALL)[0].replace('\n', '')
                        except:
                            手术经过 = ''
                    else: 
                        手术经过 = re.findall('.*手术经过(.*?)医生签名.*',operation_record, flags = re.DOTALL)[0].replace('\n', '')
                elif re.findall('失血量', operation_record) != [] and re.findall('主刀审核签名', operation_record) != []:  
                    if re.search('失血量', operation_record).start() < re.search('主刀审核签名',operation_record).start():
                        try:
                            手术经过 = re.findall('.*手术经过(.*?)失血量.*',operation_record, flags = re.DOTALL)[0].replace('\n', '')
                        except:
                            手术经过 = ''
                    else: 
                        手术经过 = re.findall('.*手术经过(.*?)主刀审核签名.*',operation_record, flags = re.DOTALL)[0].replace('\n', '')
                else:
                    手术经过 = ''

            df_ope = pd.DataFrame([])
            dict_ope = {'手术日期':手术日期, '术前诊断':术前诊断, 
                        '术中诊断':术中诊断, '实施手术':实施手术, '手术经过':手术经过
                        }
            df_ope = df_ope.append(dict_ope, ignore_index=True)
            df_record=pd.concat([df_ad, df_dis,df_ope],axis=1)

            if 住院号 == '' and 住院号_d != '':
                住院号 = 住院号_d
            elif 住院号 == '' and 住院号_d == '':
                住院号 = 住院号_o

            for index, row in df_record.iterrows():
                if row['住院号'] == '':
                    row ['住院号'] = 住院号
            #历史记录
            df_pinp = pd.DataFrame([])
            for i in range(len(list_inpatient)):
                if list_inpatient[i] == '':
                    locals()['inpatient_precord' + str(i)] = ''
                else:
                    locals()['inpatient_precord' + str(i)] = get_historical(list_inpatient[i]).strip().replace(' ', '')
        #         print('inpatient_history{index}'.format(index = i))
        #         print(locals()['inpatient_precord' + str(i)])
                if locals()['inpatient_precord' + str(i)] != '' and re.findall('姓名', locals()['inpatient_precord' + str(i)]) == []:
                    pass
                elif locals()['inpatient_precord' + str(i)] != '' and re.findall('.*?姓\xa0{0,}名[:|：]*(.{1})', locals()['inpatient_precord' + str(i)], flags = re.DOTALL)[0] == '\n':
                    try:
                        姓名_历史 = re.findall('.*?姓名[:|：]*\n{0,}(.*?)\n.*', locals()['inpatient_precord' + str(i)])[0]
                    except:
                        姓名_历史 = ''
                    if re.findall('.*院号码', locals()['inpatient_precord' + str(i)]) != []:
                        try:
                            住院号_历史 = re.findall('.*?住院号码[:|：]*\n{0,}(.*?)\n.*', locals()['inpatient_precord' + str(i)])[0]
                        except:
                            住院号_历史 = '00' + re.findall('.*?\n00(.*?)\n.*', locals()['inpatient_precord' + str(i)])[0]
                    elif re.findall('.*住院号[:|：]', locals()['inpatient_precord' + str(i)]) != []:
                        try:
                            住院号_历史 = re.findall('.*?住院号[:|：]*\n{0,}(.*?)\n.*', locals()['inpatient_precord' + str(i)])[0]
                        except:
                            住院号_历史 = ''
                    else:
                        住院号_历史 = ''
                    if re.findall('.*科室[:|：]', locals()['inpatient_precord' + str(i)]) != []:
                        科室_历史 = re.findall('.*科室[:|：]*\n{0,}(.*?)\n.*', locals()['inpatient_precord' + str(i)])[0]
                    else:
                        科室_历史 = ''
                    if re.findall('.*职业.*', locals()['inpatient_precord' + str(i)]) != []:
                        try:
                            职业_历史 = re.findall('.*职业[:|：]*\n{0,}(.*)\n.*', locals()['inpatient_precord' + str(i)])[0]
                        except:
                            职业_历史 = ''
                    else:
                        职业_历史 = ''
                    年龄_历史 = re.findall('.*年龄[:|：]*\n(.*?)\n.*', locals()['inpatient_precord' + str(i)])[0]
                    if re.findall('.*婚姻状况', locals()['inpatient_precord' + str(i)]) != []:
                        婚姻_历史 = re.findall('.*婚姻状况[:|：]*\n{0,}(.*)\n', locals()['inpatient_precord' + str(i)])[0]
                    elif re.findall('.*婚姻', locals()['inpatient_precord' + str(i)]) != [] and re.findall('.*婚姻状况', locals()['inpatient_precord' + str(i)]) == []:
                        婚姻_历史 = re.findall('.*婚姻[:|：]*\n{0,}(.*)\n', locals()['inpatient_precord' + str(i)])[0]
                    else:
                        婚姻_历史 = ''
                    if re.findall('.*[记录时间|记录日期]', locals()['inpatient_precord' + str(i)]) != []:
                        try:
                            记录日期_历史 = re.findall('.*[记录时间|记录日期][:|：]*\n{0,}(.*)\n', locals()['inpatient_precord' + str(i)])[0]
                        except:
                            记录日期_历史 =''
                    else:
                        记录日期_历史 =''
                        
                    if re.findall('.*入院时间', locals()['inpatient_precord' + str(i)]) != []:
                        try:
                            入院日期_历史 = re.findall('.*入院时间[:|：]*\n{0,}(.*)\n', locals()['inpatient_precord' + str(i)])[0]
                        except:
                            入院日期_历史 = ''
                    elif re.findall('.*入院日期', locals()['inpatient_precord' + str(i)]) != []:
                        try:
                            入院日期_历史 = re.findall('.*入院日期[:|：]*\n{0,}(.*)\n', locals()['inpatient_precord' + str(i)])[0]
                        except:
                            入院日期_历史 = ''
                    else:
                        入院日期_历史 = ''
                        
                    if re.findall('.*主诉', locals()['inpatient_precord' + str(i)]) != []:
                        try:
                            主诉_历史 = re.findall('.*主诉[:|：]*\n{0,}(.*?)\n.*', locals()['inpatient_precord' + str(i)])[0]
                        except: 
                            主诉_历史 = ''
                    else:
                        主诉_历史 = ''

                    if re.findall('.*现病史', locals()['inpatient_precord' + str(i)]) != []:
                        try:
                            现病史_历史 = re.findall('.*现病史[:|：]*\n{0,}(.*)\n', locals()['inpatient_precord' + str(i)])[0]
                        except:
                            现病史_历史 = ''
                    else:
                        现病史_历史 = ''

                    try:
                        既往史_历史 = re.findall('.*既往史[:|：]*\n{0,}(.*)\n', locals()['inpatient_precord' + str(i)])[0]
                    except:
                        既往史_历史 = ''

                    if re.findall('.*个人史', locals()['inpatient_precord' + str(i)]) != []:
                        try:
                            个人史_历史 = re.findall('.*个人史[:|：]*\n{0,}(.*)\n', locals()['inpatient_precord' + str(i)])[0]
                        except:
                            个人史_历史 = ''
                    else:
                        个人史_历史 = ''

                    if re.findall('.*婚育史', locals()['inpatient_precord' + str(i)]) != []:
                        婚育史_历史 = re.findall('.*婚育史[:|：]*\n{0,}(.*)\n', locals()['inpatient_precord' + str(i)])[0]
                    else:
                        婚育史_历史 = ''

                    if re.findall('.*家族史', locals()['inpatient_precord' + str(i)]) != []:
                        try:
                            家族史_历史 = re.findall('.*家族史[:|：]*\n{0,}(.*)\n', locals()['inpatient_precord' + str(i)])[0]
                        except:
                            家族史_历史 = ''
                    else:
                        家族史_历史 = ''

                    if re.findall('辅助检查', locals()['inpatient_precord' + str(i)]) != [] and re.findall('体格检查', locals()['inpatient_precord' + str(i)])!= []:
                        try:
                            体检_历史 = re.findall('.*体格检查(.*)辅助检查.*', locals()['inpatient_precord' + str(i)], flags = re.DOTALL)[0].replace('\n', '')
                        except:
                            体检_历史 = ''
                    elif re.findall('专科[检查|情况]', locals()['inpatient_precord' + str(i)]) != [] and re.findall('体格检查', locals()['inpatient_precord' + str(i)]) != []:
                        try:
                            体检_历史 = re.findall('.*体格检查(.*)专科[检查|情况].*', locals()['inpatient_precord' + str(i)], flags = re.DOTALL)[0].replace('\n', '')
                        except:
                            体检_历史 = ''
                    else:
                        体检_历史 = ''
                else:
                    if re.findall('姓名', locals()['inpatient_precord' + str(i)]) != []:
                        try:
                            姓名_历史= re.findall('.*?姓名[:|：](.*?)职业[:|：].*', locals()['inpatient_precord' + str(i)])[0]
                        except:
                            姓名_历史= ''
                    else:
                        姓名_历史= ''
                    if re.findall('.*科室', locals()['inpatient_precord' + str(i)]) != []:
                        科室_历史 = re.findall('.*?科室[:|：](.*?)\n.*', locals()['inpatient_precord' + str(i)])[0]
                    else:
                        科室_历史= ''
                    if re.findall('.*住院号', locals()['inpatient_precord' + str(i)]) != []:
                        住院号_历史 = re.findall('.*?住院号[:|：](.*?)\n.*', locals()['inpatient_precord' + str(i)])[0]
                    else:
                        住院号_历史= ''
                    if re.findall('.*职业', locals()['inpatient_precord' + str(i)]) != []:
                        try:
                            职业_历史 = re.findall('.*?职业[:|：](.*?)性别[:|：].*', locals()['inpatient_precord' + str(i)])[0]
                        except:
                            try :
                                职业_历史 = re.findall('.*?职业[:|：](.*?)年龄[:|：].*', locals()['inpatient_precord' + str(i)])[0]
                            except:
                                职业_历史= ''
                    else:
                        职业_历史= ''
                    if re.findall('.*年龄', locals()['inpatient_precord' + str(i)]) != []:
                        try:
                            年龄_历史 = re.findall('.*?年龄[:|：](.*?)岁住址[:|：].*', locals()['inpatient_precord' + str(i)])[0]
                        except:
                            try:
                                年龄_历史 = re.findall('.*?年龄[:|：](.*?)住址[:|：].*', locals()['inpatient_precord' + str(i)])[0]
                            except:
                                年龄_历史= ''        
                    else:
                        年龄_历史= ''
                    if re.findall('.*婚姻', locals()['inpatient_precord' + str(i)]) != []:
                        try:
                            婚姻_历史 = re.findall('.*?婚姻[:|：](.*?)供.*史.*者：.*', locals()['inpatient_precord' + str(i)], flags = re.DOTALL)[0]
                        except:
                            try:
                                婚姻_历史 = re.findall('.*?婚姻[:|：](.*?)记录日期：.*', locals()['inpatient_precord' + str(i)], flags = re.DOTALL)[0]
                            except:
                                婚姻_历史 = ''
                    else:
                        婚姻_历史 = ''
                    
                    if re.findall('.*日期', locals()['inpatient_precord' + str(i)]) != []:
                        try:
                            入院日期_历史 = re.findall('.*?入院日期[:|：]*\n{0,}(.{11})', locals()['inpatient_precord' + str(i)])[0]
                        except:
                            try:
                                入院日期_历史 = re.findall('.*?入院时间[:|：]*\n{0,}(.{11})', locals()['inpatient_precord' + str(i)])[0]
                            except:
                                入院日期_历史 = ''
                    else:
                        入院日期_历史 = ''
                    
                    if re.findall('.*记录日期', locals()['inpatient_precord' + str(i)]) != []:
                        try:
                            记录日期_历史 = re.findall('.*?记录日期[:|：]*\n{0,}(.{11})', locals()['inpatient_precord' + str(i)])[0]
                        except:
                            try:
                                记录日期_历史 = re.findall('.*?记录日期[:|：]*\n{0,}(.{9})', locals()['inpatient_precord' + str(i)])[0]
                            except:
                                记录日期_历史 = ''
                    else: 
                        记录日期_历史 = ''
                    if re.findall('.*主诉', locals()['inpatient_precord' + str(i)]) != []:
                        主诉_历史 = re.findall('.*?主诉[:|：](.*?)\n', locals()['inpatient_precord' + str(i)])[0]
                    else:
                        主诉_历史 = ''
                    if re.findall('.*现病史', locals()['inpatient_precord' + str(i)]) != []:
                        现病史_历史 = re.findall('.*现病史[:|：]*(.*)\n', locals()['inpatient_precord' + str(i)])[0]
                    else:
                        现病史_历史 = ''

                    if re.findall('.*既往史', locals()['inpatient_precord' + str(i)]) != []:
                        既往史_历史 = re.findall('.*既往史[:|：]*(.*)\n', locals()['inpatient_precord' + str(i)])[0]
                    else:
                        既往史_历史 = ''

                    if re.findall('.*个人史', locals()['inpatient_precord' + str(i)]) != []:
                        个人史_历史 = re.findall('.*个人史[:|：]*(.*)\n', locals()['inpatient_precord' + str(i)])[0]
                    else:
                        个人史_历史 = ''

                    if re.findall('.*婚育史', locals()['inpatient_precord' + str(i)]) != []:
                        婚育史_历史 = re.findall('.*婚育史[:|：]*(.*)\n', locals()['inpatient_precord' + str(i)])[0]
                    else:
                        婚育史_历史 = ''

                    if re.findall('.*家族史', locals()['inpatient_precord' + str(i)]) != []:
                        家族史_历史 = re.findall('.*家族史[:|：]*(.*)\n', locals()['inpatient_precord' + str(i)])[0]
                    else:
                        家族史_历史 = ''

                    if re.findall('.*体格检查', locals()['inpatient_precord' + str(i)]) != []:
                        try:
                            体检_历史 = re.findall('.*?体格检查(.*?)\n\n', locals()['inpatient_precord' + str(i)], flags = re.DOTALL)[0].replace('\n', '')
                        except:
                            体检_历史 = ''
                    else:
                        体检_历史 = ''
                dict_temp = {'入院日期':入院日期_历史, '主诉':主诉_历史, '姓名':姓名_历史, '科室':科室_历史, '住院号':住院号_历史, 
                                '职业':职业_历史, '年龄':年龄_历史, '婚姻':婚姻_历史, '记录日期':记录日期_历史, '现病史':现病史_历史, '既往史': 既往史_历史, 
                             '个人史': 个人史_历史, '家族史': 家族史_历史, '婚育史': 婚育史_历史, '体检':体检_历史}
                df_pinp = df_pinp.append(dict_temp, ignore_index =True)


            ##历史出院
            df_pdis = pd.DataFrame([])
            #dict_pdis = {}
            for i in range(len(list_discharge)):
                if list_discharge[i] == '':
                    locals()['discharge_precord' + str(i)] = '' 
                else:
                    locals()['discharge_precord' + str(i)] = get_historical(list_discharge[i]).strip().replace(' ', '')
        #         print('discharge_history{index}'.format(index = i))
        #         print(locals()['discharge_precord' + str(i)])
                if locals()['discharge_precord' + str(i)] != '' and re.findall('.*姓名', locals()['discharge_precord' + str(i)])!=[] and re.findall('.*?名[:|：]*(.{1})', locals()['discharge_precord' + str(i)], flags = re.DOTALL)[0]  == '\n':
                    if re.findall('.*院号码', locals()['discharge_precord' + str(i)]) != []:
                        住院号_历史d = re.findall('.*?住院号码[:|：]\n{0,}(.*?)\n.*', locals()['discharge_precord' + str(i)])[0]
                    elif re.findall('.*住院号[:|：]', locals()['discharge_precord' + str(i)]) != []:
                        住院号_历史d = re.findall('.*?住院号[:|：]\n{0,}(.*?)\n.*', locals()['discharge_precord' + str(i)])[0]
                    else:
                        住院号_历史d = ''
    
                    if re.findall('.*出院时间', locals()['discharge_precord' + str(i)]) != []:
                        try:
                            出院日期_历史 = re.findall('.*出院时间[:|：]*\n{0,}(.*)\n', locals()['discharge_precord' + str(i)])[0]
                        except:
                            出院日期_历史 = ''
                    elif re.findall('.*出院日期', locals()['discharge_precord' + str(i)]) != []:
                        try:
                            出院日期_历史 = re.findall('.*出院日期[:|：]*\n{0,}(.*)\n', locals()['discharge_precord' + str(i)])[0]
                        except:
                            出院日期_历史 = ''
                    else:
                        出院日期_历史 = ''
                    try:
                        初步诊断_历史 = re.findall('.*入院诊断[:|：]*\n{0,}(.*)\n', locals()['discharge_precord' + str(i)])[0]
                    except:
                        初步诊断_历史 = ''
                    if re.findall('.*出院时情况[:|：]', locals()['discharge_precord' + str(i)]) != []:
                        出院情况_历史 = re.findall('.*出院时情况[:|：]*\n{0,}(.*)\n', locals()['discharge_precord' + str(i)])[0]
                    elif re.findall('.*出院时情况[:|：]',locals()['discharge_precord' + str(i)]) != []:
                        出院情况_历史 = re.findall('.*出院情况[:|：]*\n{0,}(.*)\n', locals()['discharge_precord' + str(i)])[0]
                    else:
                         出院情况_历史 = ''
                    if re.findall('.*出院诊断[:|：]', locals()['discharge_precord' + str(i)]) != []:
                        try:
                            出院诊断_历史 = re.findall('.*出院诊断[:|：]*\n{0,}(.*)\n', locals()['discharge_precord' + str(i)])[0]
                        except:
                            出院诊断_历史 = ''
                    else:
                        出院诊断_历史 = ''
                    try:
                        出院医嘱_历史 = re.findall('.*出院医嘱[:|：]*\n{0,}(.*)\n', locals()['discharge_precord' + str(i)])[0]
                    except:
                        出院医嘱_历史 = ''
                else:
                    try:
                        出院日期_历史 = re.findall('.*出院日期[:|：]*\n{0,}(.*?)共住院.*', locals()['discharge_precord' + str(i)])[0]
                    except:
                        出院日期_历史 = ''
                    if re.findall('初步诊断', locals()['discharge_precord' + str(i)]) != []:
                        初步诊断_历史 = re.findall('.*初步诊断[:|：]*\n{0,}(.*?)\n', locals()['discharge_precord' + str(i)])[0]
                    else:
                        初步诊断_历史 = ''
                    if re.findall('出院情况', locals()['discharge_precord' + str(i)]) == []:
                        出院情况_历史 = ''
                    else:
                        出院情况_历史 = re.findall('.*出院情况[:|：]*\n{0,}(.*?)\n', locals()['discharge_precord' + str(i)])[0]
                    if re.findall('出院诊断', locals()['discharge_precord' + str(i)]) == []:
                        出院诊断_历史 = ''
                    else:
                        出院诊断_历史 = re.findall('.*出院诊断[:|：]*\n{0,}(.*?)\n', locals()['discharge_precord' + str(i)])[0]
                    try:
                        出院医嘱_历史 = re.findall('.*出院医嘱[:|：]*\n{0,}(.*?)\n', locals()['discharge_precord' + str(i)])[0]
                    except:
                        出院医嘱_历史 = ''
                dict_temp = {'出院日期':出院日期_历史, '初步诊断':初步诊断_历史, '出院情况':出院情况_历史, 
                                     '出院诊断':出院诊断_历史, '出院医嘱':出院医嘱_历史}
                df_pdis = df_pdis.append(dict_temp, ignore_index=True)

            df_pope = pd.DataFrame([])
            #dict_pope = {}
            for i in range(len(list_operation)):  
                if list_operation[i] == '':
                    locals()['operation_precord' + str(i)] = ''
                else:
                    locals()['operation_precord' + str(i)] = get_historical(list_operation[i]).strip().replace(' ', '')
        #         print('operation_history{index}'.format(index = i))
        #         print(locals()['operation_precord' + str(i)])
                if locals()['operation_precord' + str(i)] != '' and re.findall('.*?姓\u3000{0,}名[:|：]*(.{1})', locals()['operation_precord' + str(i)], flags = re.DOTALL)[0] == '\n':
                    if re.findall('.*院号码', locals()['operation_precord' + str(i)]) != []:
                        住院号_历史o = re.findall('.*?住院号码[:|：]\n{0,}(.*?)\n.*', locals()['operation_precord' + str(i)])[0]
                    elif re.findall('.*住院号[:|：]', locals()['operation_precord' + str(i)]) != []:
                        住院号_历史o = re.findall('.*?住院号[:|：]\n{0,}(.*?)\n.*', locals()['operation_precord' + str(i)])[0]
                    else:
                        住院号_历史o = ''
                    if locals()['operation_precord' + str(i)] != '':
                        手术日期_历史 = re.findall('.*手术日期[:|：]*\n{0,}(.*)\n', locals()['operation_precord' + str(i)])[0]
                    else:
                        手术日期_历史 = ''

                    if locals()['operation_precord' + str(i)] != '':
                        try:
                            术前诊断_历史 = re.findall('.*术前诊断[:|：]*\n{0,}(.*)\n', locals()['operation_precord' + str(i)])[0]
                        except:
                            术前诊断_历史 = ''
                    else:
                        术前诊断_历史 = ''

                    if re.findall('术中诊断', locals()['operation_precord' + str(i)]) != [] and locals()['operation_precord' + str(i)] != '':
                        try:
                            术中诊断_历史 = re.findall('.*术中诊断[:|：]*\n{0,}(.*)\n', locals()['operation_precord' + str(i)])[0]
                        except:
                            术中诊断_历史 = ''
                    else:
                        术中诊断_历史 = ''

                    if locals()['operation_precord' + str(i)] != '' and re.findall('.*实施手术', locals()['operation_precord' + str(i)]):
                        实施手术_历史 = re.findall('.*实施手术[:|：]*\n{0,}(.*)\n', locals()['operation_precord' + str(i)])[0]
                    else:
                        实施手术_历史 = ''

                    if locals()['operation_precord' + str(i)] == '':
                        手术经过_历史 = '' 
                    elif  re.findall('.*手术经过',locals()['operation_precord' + str(i)]) != []:
                        try:
                            手术经过_历史 = re.findall('.*手术经过[:|：]*\n{0,}(.*)\n',locals()['operation_precord' + str(i)])[0]
                        except:
                            手术经过_历史 = ''
                    elif re.findall('.*手术经过',locals()['operation_precord' + str(i)]) == [] and re.findall('.*手术过程', locals()['operation_precord' + str(i)])!=[]:
                        手术经过_历史 = re.findall('.*手术过程[:|：]*\n{0,}(.*)\n',locals()['operation_precord' + str(i)])[0] 
                    else:
                        手术经过_历史 = ''
                else:
                    if locals()['operation_precord' + str(i)] == '':
                        住院号_历史o = ''
                    else:
                        住院号_历史o = re.findall('.*住院号[:|：]*(.{8})', locals()['operation_precord' + str(i)])[0]
                    if locals()['operation_precord' + str(i)] == '':
                        手术日期_历史 = ''
                    else:
                        try:   
                            手术日期_历史 = re.findall('.*手术日期[:|：]*\n{0,}(.{11})', locals()['operation_precord' + str(i)])[0]
                        except:
                            try:
                                手术日期_历史 = re.findall('.*手术日期[:|：]*\n{0,}(.{9})', locals()['operation_precord' + str(i)])[0]
                            except: 
                                手术日期_历史 = ''
                    if locals()['operation_precord' + str(i)] == '' or re.findall('.*术前诊断', locals()['operation_precord' + str(i)])==[]:
                        术前诊断_历史 = ''
                    elif re.findall('.*术中诊断', locals()['operation_precord' + str(i)])!=[]:
                        try:
                            术前诊断_历史 = re.findall('.*术前诊断[:|：]*(.*)\n{0,}术中诊断：.*', locals()['operation_precord' + str(i)])[0]
                        except:
                            术前诊断_历史 = ''
                    elif re.findall('.*术中诊断', locals()['operation_precord' + str(i)])==[] and re.findall('.*术后诊断', locals()['operation_precord' + str(i)])!=[]:
                        try:
                            术前诊断_历史 = re.findall('.*术前诊断[:|：]*(.*)\n{0,}术后诊断：.*', locals()['operation_precord' + str(i)])[0]
                        except:
                            术前诊断_历史 = ''
                    else:
                        术前诊断_历史 = ''

                    if re.findall('.*术中诊断', locals()['operation_precord' + str(i)]) != [] and re.findall('.*实施手术', locals()['operation_precord' + str(i)]) != []:
                        try:
                            术中诊断_历史 = re.findall('.*术中诊断[:|：]*(.*)实施手术[:|：]*.*', locals()['operation_precord' + str(i)])[0]
                        except:
                            术中诊断_历史 = ''
                    else:
                        术中诊断_历史 = '' 

                    if locals()['operation_precord' + str(i)] == '' or re.findall('.*实施手术', locals()['operation_precord' + str(i)]) == []:
                        实施手术_历史 = ''    
                    else:
                        实施手术_历史 = re.findall('.*实施手术[:|：]*(.*)\n.*', locals()['operation_precord' + str(i)])[0]

                    if locals()['operation_precord' + str(i)] == '':
                            手术经过_历史 = ''    
                    else:
                        if re.findall('失血量', locals()['operation_precord' + str(i)]) != [] and re.findall('医生签名', locals()['operation_precord' + str(i)]) != []:
                            if re.search('失血量', locals()['operation_precord' + str(i)]).start() < re.search('医生签名',locals()['operation_precord' + str(i)]).start():
                                手术经过_历史 = re.findall('.*手术经过(.*?)失血量.*',locals()['operation_precord' + str(i)], flags = re.DOTALL)[0].replace('\n', '')
                            else: 
                                手术经过_历史 = re.findall('.*手术经过(.*?)医生签名.*',locals()['operation_precord' + str(i)], flags = re.DOTALL)[0].replace('\n', '')
                        elif re.findall('失血量', locals()['operation_precord' + str(i)]) != [] and re.findall('主刀审核签名', locals()['operation_precord' + str(i)]) != []:  
                            if re.search('失血量', locals()['operation_precord' + str(i)]).start() < re.search('主刀审核签名',locals()['operation_precord' + str(i)]).start():
                                try:
                                    手术经过_历史 = re.findall('.*手术经过(.*?)失血量.*',locals()['operation_precord' + str(i)], flags = re.DOTALL)[0].replace('\n', '')
                                except:
                                    try:
                                        手术经过_历史 = re.findall('.*实施手术(.*?)失血量.*',locals()['operation_precord' + str(i)], flags = re.DOTALL)[0].replace('\n', '')
                                    except:
                                        手术经过_历史 = ''
                            else: 
                                手术经过_历史 = re.findall('.*手术经过(.*?)主刀审核签名.*',locals()['operation_precord' + str(i)], flags = re.DOTALL)[0].replace('\n', '')
                        else:
                            手术经过_历史 = ''

                dict_temp = {'手术日期':手术日期_历史, '术前诊断':术前诊断_历史, 
                          '术中诊断':术中诊断_历史, '实施手术':实施手术_历史, '手术经过':手术经过_历史
                            }

                df_pope = df_pope.append(dict_temp, ignore_index=True)

            df_temp=pd.concat([df_pinp, df_pdis,df_pope], axis=1)
            for index, row in df_temp.iterrows():
                if row['住院号'] == '':
                    row ['住院号'] = 住院号

            df = df_record.append(df_temp, ignore_index = True)

            df.replace('', np.nan, inplace = True)
            df.dropna(axis = 0, how ='all', inplace = True, subset = ['住院号'])
            df = df.reset_index(drop = True)
            for index, row in df.iterrows():
                for date in ['入院日期','出院日期','手术日期','记录日期']:
                    try:
                        if str(row[date]).startswith('-', 4) == True:
                            if str(row[date])[7] == '-':
                                temp = row[date][:10]
                                df.loc[index,date] = datetime.datetime.strptime(temp,'%Y-%m-%d')
                            elif str(row[date])[6] == '-':
                                temp = row[date][:8]
                                df.loc[index,date] = datetime.datetime.strptime(temp,'%Y-%m-%d')
                        elif str(row[date]).startswith('年', 4) == True:
                            if str(row[date])[7] == '月':
                                temp = row[date][:10]
                                df.loc[index,date] = datetime.datetime.strptime(temp,'%Y年%m月%d')
                            elif str(row[date])[6] == '月':
                                temp = row[date][:8]
                                df.loc[index,date] = datetime.datetime.strptime(temp,'%Y年%m月%d')
                        elif str(row[date]).startswith('.', 4) == True:
                            if str(row[date])[7] == '.':
                                temp = row[date][:10]
                                df.loc[index,date] = datetime.datetime.strptime(temp,'%Y.%m.%d')
                            elif str(row[date])[6] == '.':
                                temp = row[date][:8]
                                df.loc[index,date] = datetime.datetime.strptime(temp,'%Y.%m.%d')
                    except:
                        pass
            if df.empty == False and (type(df['入院日期'][0]) == datetime.datetime) and ((datetime.datetime.now() - df['入院日期'][0]).days <= 30) and (discharge_record == ''):
                temp = open('missingdischarge.txt','a')
                temp.write(str(patient_id) + '\n')
                temp.close()
            else:
                df.to_sql(con = engine, name = 'emr',if_exists = 'append')
        except:
            out = open('b.txt','a')
            out.write(str(patient_id) + '\n')
            out.write(str(traceback.print_exc()) + '\n')
            out.close()

In [34]:
from threading import Thread, Lock
onethird = int(len(array)/3)
twothird = int(len(array)*2/3)
lock = Lock()
threads = []
t1 = Thread(target = function, args = (0, onethird))
threads.append(t1)
t2 = Thread(target = function, args = (onethird+1, twothird))
threads.append(t2)
t3 = Thread(target = function, args = (twothird+1, len(array)))
threads.append(t3)
for t in threads:
    #t.setDaemon(True)
    t.start()
for t in threads:
    t.join()

0
00219273
9676
00435412
19351
00408036
9677
00434967
19352
00447235
9678
00434938
1
00163502
9679
00434985
9680
00434940
19353
00292688
2
00227955
9681
00435003
19354
00447292
9682
00435388
19355
00447515
19356
00447358
19357
00447228
9683
00435001
3
00232706
19358
00447227
9684
00434972
19359
00447346
9685
00435053
19360
00447297
4
00350958
19361
00447342
9686
00434952
9687
00441183
19362
00447649
5
00377640
9688
00435017
9689
00434962
9690
00440609
9691
00435007
9692
00435014
6
00382508
9693
00448916
19363
00447397
7
00376539
9694
00435033
19364
00127379
9695
00437217
19365
00447347
9696
00450832
8
00439957
19366
00447350
9697
00103745
9
00453831
9698
00435041
19367
00447352
9699
00435038
10
00285547
9700
00077588
19368
00447315
19369
00447747
19370
00447317
9701
00435025
19371
00447664
11
00274812
9702
00435009
19372
00332709
9703
00434988
12
00456490
13
00381924
9704
00435012
19373
00447378
19374
00447365
9705
00435046
9706
00435034
9707
00435015
19375
00447368
9708
00435029
14
00

9909
00435292
19608
00447959
92
00349924
19609
00448083
9910
00061544
19610
00309675
9911
00445877
19611
00447645
9912
00435320
19612
00447634
9913
00435363
19613
00447648
19614
00447353
9914
00325728
19615
00447655
9915
00435272
19616
00447676
9916
00435245
19617
00449209
9917
00435291
9918
00111229
19618
00191192
93
00415851
19619
00447631
94
00252612
9919
00435244
19620
00310736
9920
00435288
19621
00447652
95
00440150
19622
00447643
9921
00435260
9922
00435543
96
00159681
19623
00448147
9923
00435297
9924
00435299
9925
00435261
19624
00447410
97
00136152
9926
00435155
19625
00447650
19626
00447630
19627
00447668
19628
00447636
9927
00435274
9928
00435258
19629
00317393
9929
00435357
9930
00435262
9931
00435230
9932
00436109
19630
00446324
9933
00435290
19631
00449894
9934
00435312
19632
00447675
9935
00435293
19633
00447620
9936
00305024
98
00382094
9937
00325616
19634
00447693
19635
00447663
19636
00442092
9938
00435255
9939
00435333
19637
00450348
9940
00436111
19638
00447572
99


19863
00449316
19864
00447987
10147
00354001
19865
00447944
10148
00435901
162
00090353
10149
00211443
19866
00447932
19867
00448072
10150
00184819
163
00172793
19868
00447786
10151
00303377
19869
00447990
19870
00203459
10152
00371212
10153
00374741
19871
00447978
10154
00313177
10155
00352464
19872
00447963
10156
00156551
164
00456643
10157
00361913
19873
00447988
19874
00447877
10158
00436027
165
00148613
19875
00448266
19876
00448689
19877
00448020
10159
00435772
19878
00455852
10160
00435703
166
00269602
19879
00448066
19880
00448597
10161
00214640
19881
00447915
167
00436745
168
00148641
19882
00448028
10162
00428277
19883
00447510
19884
00447699
10163
00455681
10164
00436008
10165
00435264
19885
00448079
10166
00435789
19886
00448125
169
00300746
19887
00448900
10167
00435615
19888
00447564
10168
00435541
19889
00448436
10169
00435598
170
00324290
19890
00448024
19891
00448035
171
00233755
10170
00435622
19892
00203120
10171
00435636
10172
00435595
19893
00459753
10173
00435604


10379
00435891
20110
00360625
10380
00435892
20111
00261370
10381
00435840
10382
00435866
10383
00436555
240
00354162
20112
00362632
10384
00456169
20113
00365905
20114
00369756
10385
00435915
10386
00435798
241
00204724
10387
00435867
20115
00369767
10388
00435949
20116
00448709
242
00149690
10389
00452116
243
00201333
10390
00435918
10391
00435806
10392
00435952
20117
00234081
10393
00454107
20118
00252011
10394
00437539
10395
00435913
244
00160497
10396
00455947
10397
00370974
20119
00243390
10398
00435926
10399
00436874
20120
00326129
10400
00435896
10401245
00382937

00435900
10402
00435893
246
00434202
247
00379423
20121
00378888
10403
00435917
10404
00455782
20122
00448671
10405
00436192
248
00433295
20123
00382350
10406
00435934
10407
00435907
10408
00435937
249
00331883
10409
00435955
20124
00448448
10410
00435957
20125
00448680
10411
00451710
20126
00255906
20127
00448363
20128
00448370
20129
00448329
20130
00231047
10412
00455574
10413
00436032
250
00234566
20131
00448345
10

10626
00436225
316
00376804
20344
00448620
10627
00366254
20345
00448534
20346
00448611
10628
00363114
20347
00234322
10629
00436239
10630
00436218
20348
00450367
10631
00435444
10632
00436199
20349
00448624
10633
00436183
10634
00436193
317
00280894
10635
00437022
20350
00448656
20351
00264305
10636
00436235
20352
00448625
318
00451708
10637
00441916
20353
00448669
319
00378027
10638
00436209
20354
00405244
20355
00448637
10639
00436189
320
00146366
10640
00436138
20356
00448648
10641
00437372
20357
00452147
10642
00436246
20358
00452965
321
00382852
10643
00436258
20359
00261258
322
00168533
20360
00293378
20361
00448677
10644
00436465
10645
00436254
20362
00448610
10646
00436266
20363
00268210
10647
00436184
20364
00448668
10648
00436854
20365
00448639
10649
00207550
20366
00449088
10650
00436253
20367
00448567
10651
00436232
10652
00436273
20368
00448618
10653
00436190
20369
00448644
323
00382351
10654
00454984
20370
00444629
10655
00441179
20371
00448686
20372
00448694
10656
00436

...

6763
00429275
6764
00446418
6765
00389263
6766
00429331
6767
00340639
6768
00345936
6769
00368100
6770
00429098
6771
00429637
6772
00429288
6773
00429314
6774
00435756
6775
00453995
6776
00425849
6777
00352691
6778
00429343
6779
00429363
6780
00429357
6781
00329894
6782
00429369
6783
00429334
6784
00429255
6785
00434855
6786
00429364
6787
00429376
6788
00289674
6789
00432908
6790
00429313
6791
00454064
6792
00429159
6793
00429419
6794
00429328
6795
00448605
6796
00429455
6797
00429456
6798
00429410
6799
00293196
6800
00226569
6801
00448120
6802
00439467
6803
00444895
6804
00442095
6805
00063146
6806
00198928
6807
00436092
6808
00429725
6809
00432789
6810
00454503
6811
00326070
6812
00434014
6813
00429524
6814
00452167
6815
00455760
6816
00437976
6817
00459212
6818
00444630
6819
00429290
6820
00429553
6821
00450097
6822
00429759
6823
00452714
6824
00222606
6825
00429375
6826
00434333
6827
00437220
6828
00273588
6829
00444779
6830
00453093
6831
00434281
6832
00436666
6833
00429595
6834
0

7349
00451554
7350
00192113
7351
00432036
7352
00432024
7353
00170987
7354
00432144
7355
00432461
7356
00432045
7357
00432034
7358
00432022
7359
00194665
7360
00436695
7361
00432046
7362
00429198
7363
00432048
7364
00432423
7365
00432050
7366
00432053
7367
00235097
7368
00432056
7369
00432082
7370
00432180
7371
00341488
7372
00241316
7373
00432268
7374
00341359
7375
00357051
7376
00217929
7377
00451827
7378
00362071
7379
00248130
7380
00366290
7381
00252533
7382
00376479
7383
00378793
7384
00127567
7385
00380716
7386
00432021
7387
00440372
7388
00432234
7389
00455654
7390
00416676
7391
00432057
7392
00244566
7393
00433441
7394
00432055
7395
00432052
7396
00432051
7397
00435492
7398
00195829
7399
00432088
7400
00432059
7401
00432954
7402
00432108
7403
00432096
7404
00432066
7405
00433139
7406
00438476
7407
00432072
7408
00432077
7409
00432067
7410
00432061
7411
00438681
7412
00432349
7413
00432193
7414
00449565
7415
00140585
7416
00432149
7417
00432115
7418
00432112
7419
00142206
7420
0

7935
00432708
7936
00433178
7937
00432748
7938
00432750
7939
00432751
7940
00432689
7941
00432756
7942
00432755
7943
00432738
7944
00432990
7945
00432670
7946
00432754
7947
00359337
7948
00335564
7949
00432582
7950
00432713
7951
00432746
7952
00432759
7953
00432765
7954
00433292
7955
00432729
7956
00432830
7957
00432637
7958
00432778
7959
00432786
7960
00432808
7961
00432788
7962
00432791
7963
00432401
7964
00432579
7965
00435839
7966
00432815
7967
00432822
7968
00432825
7969
00432839
7970
00432827
7971
00454108
7972
00442925
7973
00335154
7974
00297891
7975
00395507
7976
00361652
7977
00325396
7978
00171317
7979
00434389
7980
00434082
7981
00432918
7982
00438764
7983
00288935
7984
00432947
7985
00432977
7986
00246298
7987
00432843
7988
00444942
7989
00432847
7990
00432723
7991
00425784
7992
00432760
7993
00185531
7994
00432799
7995
00432749
7996
00432783
7997
00432835
7998
00432772
7999
00432780
8000
00432774
8001
00432850
8002
00433301
8003
00432828
8004
00432814
8005
00407638
8006
0

8521
00433479
8522
00433476
8523
00433475
8524
00433477
8525
00433482
8526
00433451
8527
00276923
8528
00160469
8529
00433496
8530
00433465
8531
00433473
8532
00433478
8533
00433487
8534
00433457
8535
00433501
8536
00436037
8537
00433026
8538
00433450
8539
00133909
8540
00433462
8541
00433536
8542
00433507
8543
00433500
8544
00433521
8545
00433464
8546
00449152
8547
00433220
8548
00433453
8549
00433515
8550
00433367
8551
00433506
8552
00359805
8553
00433481
8554
00434250
8555
00435154
8556
00433469
8557
00343417
8558
00433449
8559
00350589
8560
00433540
8561
00433542
8562
00330995
8563
00340056
8564
00204913
8565
00251741
8566
00217284
8567
00081103
8568
00356927
8569
00357538
8570
00433638
8571
00373223
8572
00433548
8573
00380481
8574
00433595
8575
00076514
8576
00434327
8577
00433546
8578
00433547
8579
00433898
8580
00433537
8581
00433530
8582
00433534
8583
00203494
8584
00433511
8585
00432663
8586
00445116
8587
00433484
8588
00433447
8589
00454501
8590
00433538
8591
00433522
8592
0

9107
00434197
9108
00452227
9109
00434238
9110
00354893
9111
00433781
9112
00434192
9113
00434282
9114
00294927
9115
00434195
9116
00434230
9117
00434254
9118
00434242
9119
00394968
9120
00434706
9121
00434299
9122
00434725
9123
00268238
9124
00434211
9125
00433349
9126
00434287
9127
00285505
9128
00434206
9129
00374049
9130
00435830
9131
00433998
9132
00434320
9133
00434296
9134
00320431
9135
00434290
9136
00434325
9137
00434264
9138
00237675
9139
00263268
9140
00434335
9141
00434316
9142
00434288
9143
00432081
9144
00434330
9145
00434304
9146
00434292
9147
00434332
9148
00434324
9149
00434293
9150
00434228
9151
00434301
9152
00434297
9153
00347191
9154
00434302
9155
00434380
9156
00434308
9157
00434352
9158
00434044
9159
00434362
9160
00434309
9161
00434280
9162
00197792
9163
00434603
9164
00434221
9165
00209410
9166
00100579
9167
00439853
9168
00434353
9169
00434348
9170
00440816
9171
00416275
9172
00365585
9173
00199370
9174
00434841
9175
00434720
9176
00434286
9177
00434372
9178
0